# Notes: You will need a dataloader 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.datasets import make_classification


In [ ]:
# Create a custom dataset
class DatasetCreator(Dataset):
    ''' CAN APPLY CUSTOM TRansform to inputw'''
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)  # Apply the transformation
        return sample, label
    

Use dataset with dataloader

In [ ]:

# Example data from sklearn
# Generate synthetic dataset
X, y = make_classification(
    n_samples=100000,  # Number of samples
    n_features=20,     # Number of features
    n_informative=15,  # Number of informative features
    n_redundant=5,     # Number of redundant features
    n_classes=5,       # Number of classes
    random_state=42    # Reproducibility
)


# Instantiate the dataset
dataset = DatasetCreator(torch.tensor(X,dtype=torch.float32), torch.tensor(y,dtype=torch.long))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## Add transformations if you want
# transform = transforms.Normalize(mean=0.5, std=0.5)  # Example normalization
# dataset = DatasetCreator(data, labels, transform=transform)
# dataloader = DataLoader(dataset, batch_size=10, shuffle=True)


Define a 1 layer network

In [ ]:
class FFN(nn.Module):
    '''
    
    :param input_size:
    :param hidden_sizes:
    :param output_size:
    :param activation: set to None to use linear model
    :param kwargs:
    '''
    def __init__(self,input_size, hidden_sizes, output_size,activation=torch.relu,**kwargs):
        super().__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activation = activation  # Store the activation function
        self.fc1 = nn.Linear(self.input_size, self.hidden_sizes[0]) #input to hidden
        self.fc2 = nn.Linear(self.hidden_sizes[0], self.output_size[0]) #hidden to output
    def forward(self,x):
        #forward pass
        x = self.fc1(x)
        if self.activation is not None:
            x = self.activation(x)
        x = self.fc2(x)
        return x

Define hyperparameters

In [ ]:
# Hyperparameters
input_size = X.shape[1]    # Number of input features
hidden_size = [20]   # Number of hidden units
output_size = [np.unique(y).shape[0]] # Number of output features 
learning_rate = 0.001
num_epochs = 100

# Create the model
model = FFN(input_size, hidden_size, output_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss() # Cross entropy  loss for classificaton
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Loop some training

In [ ]:
# Training loop

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    # Initialize counters for correct classifications
    epoch_correct = 0
    epoch_total = 0
    # Mini-batch training
    for batch_inputs, batch_labels in dataloader:
        labels= (batch_labels).long().detach()   # shape: (batch_size,)

        outputs = model(batch_inputs) # Add sequence dim
        loss = criterion(outputs, batch_labels)
            # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        predicted_class = torch.argmax(outputs, dim=1)
        # Compare predictions with labels
        correct = (predicted_class == labels).sum().item()
        epoch_correct += correct
        epoch_total += labels.size(0)
        
     # Compute average loss and accuracy for this epoch
    avg_loss = epoch_loss / len(dataloader)  # or / params['training_trials'], your choice
    accuracy = epoch_correct / epoch_total
    
    print( f"epoch: {epoch/num_epochs:.4f}, ",
          f"Loss: {avg_loss:.4f}, ",
           f"Accuracy: {accuracy:.4f}")
    
